In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
# import sys
# sys.path.append("/data/HPS/hps-analysis/Python")
import ROOT as R
import numpy as np
%jsroot on
from root_helpers import SetStyle
R.EnableImplicitMT()                      # EnableImplicitMT must be called before the RDataFrame is constructed.

Welcome to JupyROOT 6.29/01


In [2]:
import os
recompile = True
try:
    if os.path.getmtime('Particles_C.so') - os.path.getmtime('Particles.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("Particles.C++")
else:
    R.gROOT.LoadMacro("Particles_C.so")
R.Particles()

Recompile is not needed


'Particles analysis. V1.0.1 \n'

In [3]:
# Note: runs 14624 through 14673 are with Ebeam = 1.92 GeV. The rest is at 3.742 GeV
Ebeam = 3.742
ch = R.TChain("MiniDST")
#ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_014722_000*.root")
ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
#ch.Add("/data/HPS/data/physrun2021/fee/*.root")
print(f"Loaded {ch.GetEntries():,d} events ({ch.GetEntries():7.2e})")
df = R.RDataFrame(ch)

Loaded 36,587,663 events (3.66e+07)


In [4]:
print("Available data names in Tuple:")
ColumnNames=df.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if 'ecal' in str(x)]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy     ecal_cluster_hits       ecal_cluster_nhits      ecal_cluster_seed_energy
ecal_cluster_seed_index ecal_cluster_seed_ix    ecal_cluster_seed_iy    ecal_cluster_time       
ecal_cluster_x          ecal_cluster_y          ecal_cluster_z          ecal_hit_energy         
ecal_hit_index_x        ecal_hit_index_y        ecal_hit_time           part_ecal_cluster       
track_x_at_ecal         track_y_at_ecal         track_z_at_ecal         v0_em_pos_ecal_x        
v0_em_pos_ecal_y        v0_ep_pos_ecal_x        v0_ep_pos_ecal_y        

In [5]:
# Declare a local copy of the fiducial cut found in the Particle.C library, so we can easily manipulate the borders used.
R.gInterpreter.Declare("""
bool is_in_fiducial_region(int ix, int iy){
    return(
           !(ix <= -23 || ix >= 23) && /* Cut out the left and right side */
           !(iy <= -5  || iy >= 5)  && /* Cut out the top and bottom row */
           !(iy >= -1  && iy <= 1)  && /* Cut out the first row around the gap */
           !(iy >= -2  && iy <= 2   && ix >= -11 && ix <= -1) && /* Cut around the photon hole */
           !(iy >=  4  && ix >= -19 && ix <= -17) && /* Cut around crystal (-18, 5) */
          // !(iy >=  1  && iy <= 3   && ix <= -21 ) &&   /* Cut around crystal (-23, 2) */
           !(iy >= 2   && iy <= 4 && ix >= -14 && ix <= -12 ) && /* Cut around crystal ( -13, 3) */
           !(iy >=  4               && ix >= 2 && ix <= 4)  /* Cut around crystal (3, 5) */
    );
}
""")

R.gInterpreter.Declare("""
RVec<bool> fid_cut(RVec<int> ix, RVec<int> iy){
    RVec<bool> out;
    for(size_t i=0;i< ix.size();++i){
        if(
           is_in_fiducial_region(ix[i], iy[i]) &&
           ( ix[i] < 0 )  /* Only electron side */
        ){
            out.push_back(true);
        }else{
            out.push_back(false);
        }
    }
    return out;
}
""")


dfx = df.Define("n_clus","ecal_cluster_energy.size()")\
        .Define("cluster_is_fiducial","fid_cut(ecal_cluster_seed_ix,ecal_cluster_seed_iy)")\
        .Define("ecs_ix_f","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i]) out.push_back(ecal_cluster_seed_ix[i]);}; return out;")\
        .Define("ecs_iy_f","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i]) out.push_back(ecal_cluster_seed_iy[i]);}; return out;")\
        .Define("ecal_tot_energy","double etot=0; for(size_t i=0; i<ecal_hit_energy.size();++i){ etot+= ecal_hit_energy[i]; } return etot;")\
        .Define("ecal_tot_clus_e","double etot=0; for(size_t i=0; i<ecal_cluster_energy.size();++i){ etot+= ecal_cluster_energy[i]; } return etot;")\
        .Define("","")
# Calculate the ecal cluster energy for clusters with a fiducial cut (limiting the region in the ECal), and a cut on the
# number of hits in the cluster, plus a cut on the seed energy.

dfx = dfx.Define("ec_e_f_np","""
    RVec<RVec<double>> out;
    for(int n=0;n<15;n++){
        RVec<double> ntmp;
        for(size_t i=0; i<cluster_is_fiducial.size();++i){
            if( cluster_is_fiducial[i] &&
                ecal_cluster_nhits[i] == n &&
                ecal_cluster_seed_energy[i]>= 2. ){
                    ntmp.push_back(ecal_cluster_energy[i]);
            }
        }
       out.push_back(ntmp);
    }
    return out;""")

# Same, but without the cut on the number of hits.
dfx = dfx.Define("ec_e_f","""
        RVec<double> out;
        for(size_t i=0; i<cluster_is_fiducial.size();++i){
            if(cluster_is_fiducial[i]){out.push_back(ecal_cluster_energy[i]);}
        }
        return out;""")\
    .Define("ec_e_f_e","""
        RVec<double> out;
        for(size_t i=0; i<cluster_is_fiducial.size();++i){
            if(cluster_is_fiducial[i] && ecal_cluster_seed_energy[i]>= 2.){out.push_back(ecal_cluster_energy[i]);}
        }
        return out;""")


# dfx = dfx.Filter("n_clus == 1")

In [6]:
h_ecal_loc = dfx.Histo2D(("h_ecal_loc","Ecal cluster seed (ix,iy)",51,-25.5,25.5,13,-6.5,6.5),"ecal_cluster_seed_ix","ecal_cluster_seed_iy")
h_ecal_loc_fid = dfx.Histo2D(("h_ecal_loc","Ecal cluster seed (ix,iy)",51,-25.5,25.5,13,-6.5,6.5),"ecs_ix_f","ecs_iy_f")

In [7]:
#

In [8]:
n_clus = dfx.Histo1D(("n_clus", "Number of clusters", 11, -0.5, 10.5), "n_clus")
h_ecal_cluster_energy  = dfx.Histo1D(("h_ecal_cluster_energy", "Ecal Cluster Energy;E [GeV]", 300, 0., 5.), "ecal_cluster_energy")
h_ecal_cluster_energy_fid  = dfx.Histo1D(("h_ecal_cluster_energy_fid", "Ecal Cluster Energy;E [GeV]", 300, 0., 5.), "ec_e_f")
h_ecal_cluster_energy_fid_e  = dfx.Histo1D(("h_ecal_cluster_energy_fid_e", "Ecal Cluster Energy;E [GeV]", 300, 0., 5.), "ec_e_f_e")
# n_clus_1 = df_1.Histo1D(("n_clus_1","Number of clusters",11,-0.5,10.5),"n_clus")
#h_ecal_cluster_energy_1 = df_1.Histo1D(("h_ecal_cluster_energy_1","Ecal Cluster Energy;E [GeV]",300,0.,5.), "ecal_cluster_energy")
h_ecal_cluster_energy_fid_n = dfx.Filter("n_clus == 1").Histo1D(("h_ecal_cluster_energy_fid_n","Ecal Cluster Energy;E [GeV]",300,0.,5.), "ec_e_f")
h_ecal_cluster_energy_fid_n_e = dfx.Filter("n_clus == 1").Histo1D(("h_ecal_cluster_energy_fid_n_e","Ecal Cluster Energy;E [GeV]",300,0.,5.), "ec_e_f_e")

In [9]:
h_ecal_cluster_e_fid_np = []
for n in range(15):
    ecal_clus_e_with_cut = f"ec_e_f_np[{n}]"
    hist_name = f"h_ecal_cluster_e_fid_np{n}"
    h_ecal_cluster_e_fid_np.append(dfx.Define("ec_e",ecal_clus_e_with_cut).Histo1D((hist_name, "Ecal Cluster Energy;E [GeV]", 100, 0., 5.), "ec_e"))

h_ecal_cluster_nhits = dfx.Histo1D(("h_ecal_cluster_nhits","N cluster hits",21,-0.5,20.5),"ecal_cluster_nhits")
h_ecal_cluster_nhits_f = dfx.Define("nhits_f","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i]) out.push_back(ecal_cluster_nhits[i]);}; return out;")\
    .Histo1D(("h_ecal_cluster_nhits_f","N cluster hits, fid cut",21,-0.5,20.5),"nhits_f")
h_ecal_cluster_nhits_f_e = dfx.Define("nhits_f","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i] && ecal_cluster_seed_energy[i]>= 2.) out.push_back(ecal_cluster_nhits[i]);}; return out;")\
    .Histo1D(("h_ecal_cluster_nhits_f_e","N cluster hits, fid cut",21,-0.5,20.5),"nhits_f")
h_ecal_cluster_nhits_f_e_n = dfx.Filter("n_clus==1").Define("nhits_f","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i] && ecal_cluster_seed_energy[i]>= 2.) out.push_back(ecal_cluster_nhits[i]);}; return out;")\
    .Histo1D(("h_ecal_cluster_nhits_f_e","N cluster hits, fid cut",21,-0.5,20.5),"nhits_f")

h2d  = dfx.Define("ecs_ix_f_e","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i] && ecal_cluster_seed_energy[i]>= 2. && ecal_cluster_nhits[i] >= 9 ) out.push_back(ecal_cluster_seed_ix[i]);}; return out;")\
        .Define("ecs_iy_f_e","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i] && ecal_cluster_seed_energy[i]>= 2. && ecal_cluster_nhits[i] >= 9  ) out.push_back(ecal_cluster_seed_iy[i]);}; return out;")\
    .Histo2D(("h2d","Ecal cluster seed (ix,iy) w. cuts",51,-25.5,25.5,13,-6.5,6.5),"ecs_ix_f_e","ecs_iy_f_e")


In [10]:
dfx = dfx.Define("ec_e_f_np_x","""
    RVec<RVec<double>> out;
    for(int n=0;n<15;n++){
        RVec<double> ntmp;
        for(size_t i=0; i<cluster_is_fiducial.size();++i){
            if( cluster_is_fiducial[i] &&
                ecal_cluster_nhits[i] == n &&
                ecal_cluster_seed_energy[i]>= 2. &&
                !(ecal_cluster_seed_ix[i]==-8 && abs(ecal_cluster_seed_iy[i])==3) ){
                    ntmp.push_back(ecal_cluster_energy[i]);
            }
        }
       out.push_back(ntmp);
    }
    return out;""")

h_ecal_cluster_e_fid_np_x = []
for n in range(15):
    ecal_clus_e_with_cut = f"ec_e_f_np_x[{n}]"
    hist_name = f"h_ecal_cluster_e_fid_np_x{n}"
    h_ecal_cluster_e_fid_np_x.append(dfx.Define("ec_e_x",ecal_clus_e_with_cut).Histo1D((hist_name, "Ecal Cluster Energy with cut;E [GeV]", 100, 0., 5.), "ec_e_x"))


In [11]:
c0 = R.TCanvas("c0","Canvas1",1000,900)
c0.Divide(1,2)
p1 = c0.cd(1)
p1.SetLogz(1)
h_ecal_loc.SetStats(0)
h_ecal_loc.Draw("colz")
p2 = c0.cd(2)
p2.SetLogz(1)
h_ecal_loc_fid.SetStats(0)
h_ecal_loc_fid.Draw("colz")
c0.Draw()

In [15]:
c1 = R.TCanvas("c1","Canvas1",1200,700)
R.gStyle.SetOptStat(0)
c1.Divide(2,1)
p1 = c1.cd(1)
p1.SetLogy(1)
h_ecal_cluster_energy.SetLineColor(R.kBlue)
h_ecal_cluster_energy.Draw()
#h_ecal_cluster_energy_1.SetLineColor(R.kRed)
#h_ecal_cluster_energy_1.Draw("same")
h_ecal_cluster_energy_fid.SetLineColor(R.kGreen)
h_ecal_cluster_energy_fid.Draw("same")
h_ecal_cluster_energy_fid_e.SetLineColor(R.kRed)
h_ecal_cluster_energy_fid_e.Draw("same")
h_ecal_cluster_energy_fid_n.SetLineColor(R.kViolet)
h_ecal_cluster_energy_fid_n.Draw("same")
h_ecal_cluster_energy_fid_n_e.SetLineColor(R.kOrange)
h_ecal_cluster_energy_fid_n_e.Draw("same")
leg1 = R.TLegend(0.65,0.7,0.9,0.9)
leg1.AddEntry(h_ecal_cluster_energy.GetValue(), "No Cuts")
leg1.AddEntry(h_ecal_cluster_energy_fid.GetValue(),"Fiducial Cuts")
leg1.AddEntry(h_ecal_cluster_energy_fid_e.GetValue(),"F.C., Eseed>2")
leg1.AddEntry(h_ecal_cluster_energy_fid_n.GetValue(),"F.C., 1 cluster")
leg1.AddEntry(h_ecal_cluster_energy_fid_n_e.GetValue(),"F.C, Es>2, 1 clus.")
leg1.Draw()
p2 = c1.cd(2)
p2.SetLogy(0)
h_ecal_cluster_energy_fid_n_e_copy =  h_ecal_cluster_energy_fid_n_e.GetValue().Clone("h_ecal_cluster_energy_fid_n_e_copy")
h_ecal_cluster_energy_fid_n_e_copy.Scale(7)
h_ecal_cluster_energy_fid_e.Draw()
h_ecal_cluster_energy_fid_n_e_copy.Draw("same")
leg1_2 = R.TLegend(0.1,0.75,0.4,0.9)
leg1_2.AddEntry(h_ecal_cluster_energy_fid_e.GetValue(),"F.C., Eseed>2")
leg1_2.AddEntry(h_ecal_cluster_energy_fid_n_e.GetValue(),"F.C, Es>2, 1 clus x7")
leg1_2.Draw()
#h_ecal_cluster_energy_1_fid.SetLineColor(R.kOrange)
#h_ecal_cluster_energy_1_fid.Draw("same")
c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [13]:
c2 = R.TCanvas("c2","Canvas1",1200,600)
c2.Divide(2,1)
p1 = c2.cd(1)
p1.SetLogy(0)
line_colors = [R.kBlack,   R.kBlue,   R.kGreen,   R.kOrange,   R.kRed,   R.kTeal,   R.kViolet,
               R.kGray,    R.kBlue+1, R.kGreen+1, R.kOrange+1, R.kRed+1, R.kTeal+1, R.kViolet+1,
               R.kGray+1,  R.kBlue+2, R.kGreen+2, R.kOrange+2, R.kRed+2, R.kTeal+2, R.kViolet+2]
leg2 = R.TLegend(0.75,0.6,0.9,0.9)
max=0
ii=0
for i in range(len(h_ecal_cluster_e_fid_np)):
    if h_ecal_cluster_e_fid_np[i].GetMaximum() > max:
        max = h_ecal_cluster_e_fid_np[i].GetMaximum()
        ii = i
h_ecal_cluster_e_fid_np[ii].Draw()
for i in range(3,min(14,len(h_ecal_cluster_e_fid_np))):
    h_ecal_cluster_e_fid_np[i].SetStats(0)
    h_ecal_cluster_e_fid_np[i].SetLineWidth(2)
    h_ecal_cluster_e_fid_np[i].SetLineColor(line_colors[i])
    h_ecal_cluster_e_fid_np[i].Draw("same")
    leg2.AddEntry(h_ecal_cluster_e_fid_np[i].GetValue(), f"n=={i}")
leg2.Draw()

p2 = c2.cd(2)
p2.SetLogy(1)
h_ecal_cluster_nhits.Draw()
h_ecal_cluster_nhits_f.SetLineColor(R.kRed)
h_ecal_cluster_nhits_f.Draw("same")
leg2_2 = R.TLegend(0.75,0.6,0.9,0.9)
leg2_2.AddEntry(h_ecal_cluster_nhits,"No cuts")
leg2_2.AddEntry(h_ecal_cluster_nhits_f,"Fid Cuts")

c2.Draw()

In [14]:
c4 = R.TCanvas("c4","Canvas4",900,450)
h2d.Draw("colz")
c4.Draw()

In [15]:
hh = h_ecal_cluster_e_fid_np[9].GetValue()
hh2 = h_ecal_cluster_e_fid_np[8].GetValue()
c3 = R.TCanvas("c3","Canvas3",700,700)
hh.Draw()
hh2.Draw("same")
c3.Draw()

In [16]:
# h2d  = dfx.Define("ecs_ix_f_e","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i] && ecal_cluster_seed_energy[i]>= 2. && ecal_cluster_nhits[i] >= 10 && ecal_cluster_nhits[i] <= 12 ) out.push_back(ecal_cluster_seed_ix[i]);}; return out;")\
#         .Define("ecs_iy_f_e","RVec<int> out;for(size_t i=0; i<cluster_is_fiducial.size();++i){ if(cluster_is_fiducial[i] && ecal_cluster_seed_energy[i]>= 2. && ecal_cluster_nhits[i] >= 9  ) out.push_back(ecal_cluster_seed_iy[i]);}; return out;")\
#     .Histo2D(("h2d","Ecal cluster seed (ix,iy) w. cuts",51,-25.5,25.5,13,-6.5,6.5),"ecs_ix_f_e","ecs_iy_f_e")


In [17]:
hh3 = h_ecal_cluster_e_fid_np_x[9].GetValue()
hh4 = h_ecal_cluster_e_fid_np_x[8].GetValue()
c5 = R.TCanvas("c5","Canvas3",700,700)
hh.Draw()
hh2.Draw("same")
hh3.Draw("same")
hh4.Draw("same")
c5.Draw()